In [1]:
!pip install -q -U transformers accelerate git+https://github.com/huggingface/peft.git

In [2]:
!pip install -q datasets bitsandbytes einops wandb loralib --upgrade --quiet evaluate

In [3]:
#conda install -c pytorch torchvision cudatoolkit=10.1 pytorch

In [4]:
import bitsandbytes as bnb


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [5]:
from datasets import load_dataset

In [6]:
import torch
import torch.nn as neuralNetwork

In [7]:
import transformers 
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [8]:
!pip install peft

In [9]:
from peft import (
    #prepare_model_for_kbit_training,
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
)

In [10]:
RANDOM_SEED = 42
transformers.set_seed(RANDOM_SEED)
CUTOFF_LEN = 700
OUTPUT_DIR = '/kaggle/working/'

In [11]:
loraConfig = LoraConfig(
    r=8,
    lora_alpha=16, 
    target_modules=['q_proj','v_proj', 'o_proj', 'k_proj'], 
    lora_dropout=0.05,
    bias='none', 
    task_type='CAUSAL_LM',
)

In [12]:
bnbConfig = BitsAndBytesConfig(
    load_in_4bit=True, # <<< used to enable 4-bit quantization by replacing the Linear layers with FP4/NF4 layers.
    bnb_4bit_use_double_quant=True, # <<< Enable double quantization = to 4bit, since normal quantization is 8bit.
    bnb_4bit_quant_type="nf4", # <<< The type of quantization which is nf4 or fp4
    bnb_4bit_compute_dtype=torch.float16 # <<< The computation type, Note: bfloat is for A100 gpu and above.
)

In [13]:
!pip install --upgrade accelerate

In [14]:
model = LlamaForCausalLM.from_pretrained('huggyllama/llama-7b',
                                         load_in_4bit=True,
                                         torch_dtype=torch.bfloat16, #torch.float16, 
                                         quantization_config=bnbConfig,
                                         device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
model.gradient_checkpointing_enable()

In [27]:
model = prepare_model_for_int8_training(model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [28]:
model = get_peft_model(model, loraConfig)

In [29]:
import wandb
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [30]:
wandb_key = user_secrets.get_secret("wandb_api_key")
#("f9d1692baeae771a099538946b1c7cf9e40b6db5")
wandb.login(key=wandb_key) 

wandb: Currently logged in as: jaiswaldelhi1. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [31]:
hf_key = user_secrets.get_secret("huggingFaceToken") # <<< huggingface key saved here.

In [32]:
dataset = load_dataset("0sunfire0/QA-Treasuries-539")

  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.

### Instruction:
{data_point["Questions"]}

### Response:
{data_point["Answers"]}"""


In [34]:
tokenizer = LlamaTokenizer.from_pretrained('huggyllama/llama-7b')
tokenizer.pad_token_id = 1

In [38]:
dataset = dataset.shuffle().map(
    lambda data_point: tokenizer(
        generate_prompt(data_point),
        truncation=True, # <<< Enable the max_length parameter below.
        max_length=CUTOFF_LEN, # <<< The max sequence length the tokenizer can use (was defind above to 256)
        padding="max_length", # <<< The max padding the tokenizer can use.
    )
)

Map:   0%|          | 0/539 [00:00<?, ? examples/s]

In [39]:
trainer = transformers.Trainer(
    model=model, # <<< Use the pretrained model we loaded earlier.
    train_dataset=dataset["train"], # <<< Use the dataset we loaded earlier.
    tokenizer=tokenizer, # <<< Use the pretrained tokenizerwe loaded earlier.
    args=transformers.TrainingArguments(
        seed=RANDOM_SEED, # <<< Random seed that will be set at the beginning of training, was defined earlier.
        data_seed=RANDOM_SEED, # <<< Random seed to be used with data samplers. 
        fp16=True, # <<< Use fp16 16-bit (mixed) precision training instead of 32-bit(for A100 gpu use bf16 instead).
        #bf16=True,
        output_dir=OUTPUT_DIR, # <<< Output directory where the model weights will be saved also the check points.
        num_train_epochs=17, # Number of training epochs(better than using steps only since it run on all the data)
        warmup_ratio=0.05, # <<<  Ratio of total training steps used for a linear warmup from 0 to learning_rate.
        auto_find_batch_size=True, # <<< Find the optimal batch size default is 8, it works well on free gpu, (For higher gpu define per_device_train_batch_size with 32 and the auto_find_batch_size will lower it in case vram overload)
        learning_rate=2e-5, # <<< the learning rate (0.0002) for higher gpu use (2e-5) for more training stability.
        weight_decay=0.1, # <<< Help fit the model, Default is 0.01 (used by fastai) but 0.1 works well.
        logging_steps=135, # <<< Output the training/test loss every how many steps.
        optim="paged_adamw_8bit", # <<< the optimizer for adjusting model parameters to reduce model error in each training step.
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False), # Clean and organise the data inputs in training.   
)

In [40]:
model.config.use_cache = False

In [ ]:
##if torch.__version__ >= '2' and sys.platform != 'win32':
  ##  model = torch.compile(model)

In [ ]:
# while importing from peft used prepare_model_for_int8_training instead of prepare_model_for_kbit_training
# similarly while initializing model, called method prepare_model_for_int8_training() instead of prepare_model_for_kbit_training
# 

In [41]:
trainer.train()

Step,Training Loss
135,0.923100
270,0.640800
405,0.561900
540,0.492300
675,0.425600
810,0.363700
945,0.304000
1080,0.256000


TrainOutput(global_step=1156, training_loss=0.47833663617038397, metrics={'train_runtime': 38188.1751, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.03, 'total_flos': 1.299905701011456e+17, 'train_loss': 0.47833663617038397, 'epoch': 17.0})

In [ ]:
wandb.finish()

In [ ]:
model.save_pretrained(OUTPUT_DIR)

In [46]:
model.push_to_hub("Llama_train_sk", use_auth_token=hf_key)

adapter_model.bin:   0%|          | 0.00/33.7M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/SKumari/Llama_train_sk/commit/6885f877956d5663a1434ca44519116ac7247767', commit_message='Upload model', commit_description='', oid='6885f877956d5663a1434ca44519116ac7247767', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
#Decreased learning rate.
#Updated attention blocks.
#Load the model in 4bit and float16 to save vram.
#Increased warmup ratio.